In [1]:
import pandas as pd
import numpy as np
import time
import os
import progressbar

taxonomy = pd.read_excel('./taxonomy/gene_catalog_taxonomy.xlsx')
ardb = pd.read_excel('./ARDB/gene_catalog_ardb_annotation.xlsx')

#修改lsts中内容
lsts = ['AC1','AC2','AC3']
annotations = ['integrase','IS','plasmid','RepBase']

In [2]:
import warnings
warnings.filterwarnings("ignore")
widgets = [
     '匹配: ', progressbar.Percentage(),  # 进度条标题
     ' ', progressbar.Bar(marker='#', left='[', right=']', fill=' '),  # 进度条填充、边缘字符
     ' ', progressbar.Timer(),  # 已用的时间
     ' ', progressbar.ETA(),  # 剩余时间
     ' ', progressbar.FileTransferSpeed(),  # 下载速度
]

In [3]:
def Split_Excel_Data(excel_path, name_data,folder='PROCESS'):
    # 将excel的数据切割开，按照做实验中gene顺序分成四个文件
    widgets = [
     '分割: ', progressbar.Percentage(),  # 进度条标题
     ' ', progressbar.Bar(marker='#', left='[', right=']', fill=' '),  # 进度条填充、边缘字符
     ' ', progressbar.Timer(),  # 已用的时间
     ' ', progressbar.ETA(),  # 剩余时间
     ' ', progressbar.FileTransferSpeed(),  # 下载速度
    ]
    Excel_data = pd.read_excel(excel_path)
    Excel_data['num_G'] = Excel_data['#Gene'].map(lambda x:x.split('_')[0])
    bar = progressbar.ProgressBar(widgets = widgets)
    for i in bar(Excel_data['num_G'].unique()):
        Excel_data_i = Excel_data.loc[Excel_data['num_G'] == i]
        Excel_data_i.drop('num_G',axis=1)
        excel_name = '{}_'.format(i) + name_data + '.csv'
        Excel_data_i[['#Gene']].to_csv('./{}/{}'.format(folder,excel_name))
    

def ardb_orf_get_fas(i):
    with open('./ORF/{}.orf.ffn'.format(i), 'r+') as o:
        orf = o.readlines()
    with open('./CONTIG/{}.assembly.fas'.format(i), 'r+') as f:
        fas = f.readlines()
    fas_data = pd.DataFrame(fas[1::2], columns=['sequence'])
    fas_data['fas'] = np.array(fas[0::2])
    fas_data['fas'] = fas_data.fas.map(lambda x: x.split(' ')[0].split('>')[1])

    orf_data = pd.DataFrame(orf[1::2], columns=['sequence'])
    orf_data['#Gene'] = np.array(orf[0::2])
    orf_data['#Gene'] = orf_data['#Gene'].map(lambda x: x.split(' ')[0].split('>')[1].split('\n')[0])
    orf_data['sequence'] = orf_data['sequence'].map(lambda x: x.split('\n')[0])
    ardb_data = pd.read_csv('./ARDB/{}_ardb.csv'.format(i))[['#Gene']]
    ardb_data_orf = pd.merge(ardb_data,orf_data,on='#Gene',how='left')
    ardb_data_orf['fas'] = -1
    ardb_data_orf['p_or_n'] = 1
    ardb_data_orf['sequence_n'] = 0
    ardb_data_orf['sequence_n'] = ardb_data_orf['sequence'].map(lambda x:x[-1::-1])
    ardb_data_orf['sequence_n'] = ardb_data_orf['sequence_n'].map(lambda x:x.replace('A','1').replace('T','2').replace('G','3').replace('C','4'))
    ardb_data_orf['sequence_n'] = ardb_data_orf['sequence_n'].map(lambda x:x.replace('1','T').replace('2','A').replace('3','C').replace('4','G'))

    s = ''.join(fas_data['sequence'].tolist())
    bar = progressbar.ProgressBar(widgets = widgets,maxval=len(ardb_data_orf['sequence']))
    bar.start()
    for j,val in enumerate(ardb_data_orf['sequence']):
        if val in s:
            ardb_data_orf['fas'].loc[j] = fas_data['fas'].loc[s[:s.find(val)].count('\n')]
        elif ardb_data_orf['sequence_n'].loc[j] in s:
            ardb_data_orf['fas'].loc[j] = fas_data['fas'].loc[s[:s.find(ardb_data_orf['sequence_n'].loc[j])].count('\n')]
            ardb_data_orf['p_or_n'].loc[j] = -1
        bar.update(j)
    bar.finish()

    ardb_data_orf['orf_begin'] = -1
    ardb_data_orf['orf_end'] = -1
    for j,val in enumerate(ardb_data_orf['sequence']):
        if ardb_data_orf['p_or_n'].loc[j] == 1:
            orf_sequence = val
            ardb_data_orf['orf_begin'].loc[j] = fas_data['sequence'].loc[fas_data['fas']==ardb_data_orf['fas'].loc[j]].values[0].find(orf_sequence)   
        elif ardb_data_orf['p_or_n'].loc[j] == -1:
            orf_sequence = ardb_data_orf['sequence_n'].loc[j]
            ardb_data_orf['orf_begin'].loc[j] = fas_data['sequence'].loc[fas_data['fas']==ardb_data_orf['fas'].loc[j]].values[0].find(orf_sequence)   
        ardb_data_orf['orf_end'].loc[j] = ardb_data_orf['orf_begin'].loc[j] + len(orf_sequence)
    ardb_data_orf.to_csv('./PROCESS/{}_orf_fas.csv'.format(i))   
    
def all_orf_get_fas(i,folder_name='taxonomy',filename='all'):
    with open('./ORF/{}.orf.ffn'.format(i), 'r+') as o:
        orf = o.readlines()
    with open('./CONTIG/{}.assembly.fas'.format(i), 'r+') as f:
        fas = f.readlines()
    fas_data = pd.DataFrame(fas[1::2], columns=['sequence'])
    fas_data['fas'] = np.array(fas[0::2])
    fas_data['fas'] = fas_data.fas.map(lambda x: x.split(' ')[0].split('>')[1])
    orf_data = pd.DataFrame(orf[1::2], columns=['sequence'])
    orf_data['#Gene'] = np.array(orf[0::2])
    orf_data['#Gene'] = orf_data['#Gene'].map(lambda x: x.split(' ')[0].split('>')[1].split('\n')[0])
    orf_data['fas'] = -1
    orf_data.dropna(inplace=True)
    orf_data['sequence'] = orf_data['sequence'].map(lambda x:x.split('\n')[0])
    orf_data['p_or_n'] = 1
    orf_data['sequence_n'] = 0
    orf_data['sequence_n'] = orf_data['sequence'].map(lambda x:x[-1::-1])
    orf_data['sequence_n'] = orf_data['sequence_n'].map(lambda x:x.replace('A','1').replace('T','2').replace('G','3').replace('C','4'))
    orf_data['sequence_n'] = orf_data['sequence_n'].map(lambda x:x.replace('1','T').replace('2','A').replace('3','C').replace('4','G'))
    
    data = pd.read_csv('./PROCESS/{}_orf_fas.csv'.format(i))
    data = data.drop(['Unnamed: 0'], axis=1)
    data = data.loc[data['fas'] != '-1']
    fas = pd.merge(pd.DataFrame(data.fas.unique(), columns=['fas']), fas_data, on='fas', how='left')
    s = ''.join(fas['sequence'].tolist())
    bar = progressbar.ProgressBar(widgets = widgets,maxval=len(orf_data['sequence']))
    bar.start()
    for j,val in enumerate(orf_data['sequence']):
        if val in s:
            orf_data['fas'].loc[j] = fas['fas'].loc[s[:s.find(val)].count('\n')]
        elif orf_data['sequence_n'].loc[j] in s:
            orf_data['fas'].loc[j] = fas['fas'].loc[s[:s.find(orf_data['sequence_n'].loc[j])].count('\n')]
            orf_data['p_or_n'].loc[j] = -1
        bar.update(j)
    bar.finish()
    orf_data['orf_begin'] = -1
    orf_data['orf_end'] = -1
    for j,val in enumerate(orf_data['sequence']):
        if orf_data['fas'].loc[j] != -1 and orf_data['fas'].loc[j] != '-1':
            if orf_data['p_or_n'].loc[j] == 1:
                orf_sequence = val
                orf_data['orf_begin'].loc[j] = fas['sequence'].loc[fas['fas']==orf_data['fas'].loc[j]].values[0].find(orf_sequence)   
                if orf_data['orf_begin'].loc[j] == -1:
                    print(j)
            elif orf_data['p_or_n'].loc[j] == -1:
                orf_sequence = orf_data['sequence_n'].loc[j]
                orf_data['orf_begin'].loc[j] = fas['sequence'].loc[fas['fas']==orf_data['fas'].loc[j]].values[0].find(orf_sequence)
                if orf_data['orf_begin'].loc[j] == -1:
                    print(orf_sequence)
            orf_data['orf_end'].loc[j] = orf_data['orf_begin'].loc[j] + len(orf_sequence)
    orf_data = orf_data.loc[orf_data['fas'] != -1]
    orf_data = orf_data.loc[orf_data['fas'] != '-1']
    orf_data.to_csv('./PROCESS/{}_all_fas.csv'.format(i,filename))

def merge_data(i,annotations):
    orf_fas = pd.read_csv('./PROCESS/{}_orf_fas.csv'.format(i))
    orf_fas = orf_fas.drop(['Unnamed: 0'],axis=1)
    ardb_data = pd.merge(orf_fas,ardb,on='#Gene')
    all_fas = pd.read_csv('./PROCESS/{}_all_fas.csv'.format(i))
    all_fas = all_fas.drop(['Unnamed: 0'],axis=1)
    for annotation in annotations:
        mge = pd.read_excel('./MGE/{}_annotation.xlsx'.format(annotation))
        mge_data = pd.merge(mge,all_fas,on='#Gene')
        mge_data.drop(['sequence','sequence_n'],axis=1,inplace=True)
        mge_data = mge_data.loc[mge_data['fas'] != '-1']
        mge_data = mge_data.loc[mge_data['fas'] != -1]
        mge_data = mge_data.rename(columns = {'#Gene':'{}_Gene'.format(annotation),
                                              'orf_begin':'{}_orf_begin'.format(annotation),
                                              'orf_end':'{}_orf_end'.format(annotation),
                                              'p_or_n':'{}_p_or_n'.format(annotation)})
        ardb_data = pd.merge(ardb_data,mge_data,on='fas',how='left')
    taxonomy_data = pd.read_csv('./taxonomy/{}_taxonomy.csv'.format(i))
    taxonomy_data = taxonomy_data.drop(['Unnamed: 0'],axis=1)
    taxonomy_data = pd.merge(taxonomy_data,all_fas,on='#Gene')
    taxonomy_data = pd.merge(taxonomy_data,taxonomy,on='#Gene',how='left')
    taxonomy_data.drop(['sequence','sequence_n'],axis=1,inplace=True)
    taxonomy_data = taxonomy_data.rename(columns = {'#Gene':'taxonomy_Gene',
                                              'orf_begin':'taxonomy_orf_begin',
                                              'orf_end':'taxonomy_orf_end',
                                              'p_or_n':'taxonomy_p_or_n'})
    taxonomy_data = taxonomy_data.loc[taxonomy_data.fas != -1]
    taxonomy_data = taxonomy_data.loc[taxonomy_data.fas != '-1']
    taxonomy_data['count'] = taxonomy_data['Species']
    taxonomy_count = pd.DataFrame(taxonomy_data.groupby(['fas','Species']).agg({'count': 'count'}))
    taxonomy_count = taxonomy_count.reset_index()
    taxonomy_count = pd.merge(taxonomy_count, taxonomy_count.groupby('fas', as_index=False)['count'].sum(), on='fas', how='left')
    taxonomy_count.columns = ['fas','Species','count','sum']
    taxonomy_count['avg'] = taxonomy_count['count'] / taxonomy_count['sum']
    taxonomy_count_data = pd.merge(taxonomy_count.loc[taxonomy_count.avg > 0.5], taxonomy_data, on=['fas', 'Species'], how='left')
    taxonomy_count_data = taxonomy_count_data.drop(['count_x', 'count_y', 'sum', 'avg'], axis=1)
    ardb_data = pd.merge(ardb_data,taxonomy_count_data,on='fas',how='left')
    ardb_data.drop(['sequence','sequence_n'],axis=1,inplace=True)
    ardb_data.to_csv('./final/ardb_data_{}.csv'.format(i))

In [4]:
Split_Excel_Data('./taxonomy/gene_catalog_taxonomy.xlsx',name_data='taxonomy',folder='taxonomy')
Split_Excel_Data('./ARDB/gene_catalog_ardb_annotation.xlsx',name_data='ardb',folder='ARDB')

分割: 100% [####################] Elapsed Time: 0:00:00 Time: 0:00:00 229.36  B/s
分割: 100% [####################] Elapsed Time: 0:00:00 Time: 0:00:00 288.98  B/s


In [5]:
from concurrent.futures import ProcessPoolExecutor
# with ProcessPoolExecutor(max_workers=1) as executor:
#     futures = [executor.submit(ardb_orf_get_fas,lst)
#               for lst in lsts]
# # print('成功完成匹配')
for lst in lsts:
    ardb_orf_get_fas(lst)
    all_orf_get_fas(lst)
    merge_data(lst,annotations)
    print('合并成功'.format(lst))
    print('-'*50)

匹配: 100% [####################] Elapsed Time: 0:16:21 Time: 0:16:21   0.79  B/s
匹配: 100% [####################] Elapsed Time: 6:54:27 Time: 6:54:27  17.64  B/s


合并成功
--------------------------------------------------


匹配: 100% [####################] Elapsed Time: 0:19:11 Time: 0:19:11   0.71  B/s
匹配: 100% [####################] Elapsed Time: 7:42:27 Time: 7:42:27  20.13  B/s


合并成功
--------------------------------------------------


匹配: 100% [####################] Elapsed Time: 0:13:16 Time: 0:13:16   0.79  B/s
匹配: 100% [####################] Elapsed Time: 4:11:46 Time: 4:11:46  31.73  B/s


合并成功
--------------------------------------------------


In [6]:
for i in lsts:
    ardb_data = pd.read_csv('./final/ardb_data_{}.csv'.format(i))
    ardb_data = ardb_data.drop('Unnamed: 0',axis=1)
    ardb_data = ardb_data.loc[ardb_data.fas!=-1]
    ardb_data = ardb_data.loc[ardb_data.fas!='-1']
    ardb_data.to_csv('./final/not_null/ardb_data_not_null_{}.csv'.format(i))
    
    ardb_data_species = ardb_data.loc[ardb_data.Species.notnull()]
    ardb_data_species.to_csv('./final/species/ardb_data_species_{}.csv'.format(i))
    
    ardb_data_mge = pd.DataFrame(columns = ardb_data.columns)
    for j in annotations:
        if '{}_Gene'.format(j) in ardb_data.columns:
            ardb_data_mge = pd.concat([ardb_data_mge,ardb_data.loc[ardb_data['{}_Gene'.format(j)].notnull()]],axis=0)
    ardb_data_mge.to_csv('./final/mge/ardb_data_mge_{}.csv'.format(i))
    
    ardb_data = pd.read_csv('./final/ardb_data_{}.csv'.format(i))
    ardb_data.drop('Unnamed: 0',axis=1,inplace=True)
    ardb_data_more_2 = pd.DataFrame(ardb_data.fas.value_counts())
    ardb_data_more_2 = ardb_data_more_2.loc[ardb_data_more_2.fas>=2].reset_index()
    ardb_data_more_2.columns = ['fas','count']
    ardb_data_more_2.drop('count',axis=1,inplace=True)
    ardb_data_more = pd.merge(ardb_data_more_2,ardb_data,on='fas',how='left')
    
    ardb_data_more.to_csv('./final/more_2_ardb/ardb_data_more_2_{}.csv'.format(i))

In [7]:
for i in lsts:
    all_fas = pd.read_csv('./PROCESS/{}_all_fas.csv'.format(i))
    ardb_data = pd.read_csv('./final/ardb_data_{}.csv'.format(i))
    all_fas = all_fas.drop(['Unnamed: 0', 'sequence', 'sequence_n'], axis = 1)
    ardb_data = ardb_data.drop(['Unnamed: 0'], axis = 1)
    ardb_data = ardb_data.rename(columns = {'#Gene':'ardb_Gene',
                                              'orf_begin':'ardb_orf_begin',
                                              'orf_end':'ardb_orf_end',
                                              'p_or_n':'ardb_p_or_n'})
    all_fas = pd.merge(all_fas, ardb_data, on='fas')
    all_fas.to_csv('./final/all_fas_{}.csv'.format(i))

In [8]:
all_fas

,#Gene,fas,p_or_n,orf_begin,orf_end,ardb_Gene,ardb_p_or_n,ardb_orf_begin,ardb_orf_end,SARG,...,taxonomy_orf_begin,taxonomy_orf_end,taxonomy_Gene,Kingdom,Phylum,Class,Order,Family,Genus,Species
0,AC3_orf_997,k141_396721,1,1,454,AC3_orf_997,1,1,454,bacitracin__bacA_train_msa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AC3_orf_998,k141_396721,-1,510,954,AC3_orf_997,1,1,454,bacitracin__bacA_train_msa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AC3_orf_2000,k141_170054,-1,2,263,AC3_orf_2028,1,30173,31910,multidrug__multidrug_ABC_transporter_train_msa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AC3_orf_2001,k141_170054,-1,369,3882,AC3_orf_2028,1,30173,31910,multidrug__multidrug_ABC_transporter_train_msa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AC3_orf_2002,k141_170054,-1,3871,5623,AC3_orf_2028,1,30173,31910,multidrug__multidrug_ABC_transporter_train_msa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5274,AC3_orf_476324,k141_169969,1,1,439,AC3_orf_476326,-1,807,2169,multidrug__mdtK_train_msa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5275,AC3_orf_476325,k141_169969,-1,468,771,AC3_orf_476326,-1,807,2169,multidrug__mdtK_train_msa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5276,AC3_orf_476326,k141_169969,-1,807,2169,AC3_orf_476326,-1,807,2169,multidrug__mdtK_train_msa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5277,AC3_orf_476327,k141_169969,-1,2172,3234,AC3_orf_476326,-1,807,2169,multidrug__mdtK_train_msa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
